# Equity Options Fundamentals

This notebook demonstrates how to access and use the functinalities of **Equity Options** which are part of our **QPS** module within LSEG Financial Analytics SDK.

## Imports

Import the following necessary modules.

This notebook uses the external library **pandas**; please ensure it is installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments.options import (
    value,
    OptionDefinition,
    OptionDefinitionInstrument,
    OptionPricingParameters,
    UnderlyingDefinition,
    ExerciseDefinition,
    ScheduleDefinition,
)

from lseg_analytics.pricing.common import AdjustableDate
from datetime import date
import pandas as pd
import json as js


## Creating an Equity Option Definition

In [2]:
definition = OptionDefinition(
    underlying = UnderlyingDefinition(
        code="ASML.AS",
        underlying_type="Equity"
    ),
    exercise = ExerciseDefinition(
        strike=150,
        exercise_style="European",
        schedule = ScheduleDefinition(
            end_date = AdjustableDate(
                date = date(2025, 9, 13)
            )
        )
    ),
    option_type="Call"
)

print(js.dumps(definition.as_dict(), indent=4))

{
    "underlying": {
        "code": "ASML.AS",
        "underlyingType": "Equity"
    },
    "exercise": {
        "strike": 150,
        "exerciseStyle": "European",
        "schedule": {
            "endDate": {
                "dateType": "AdjustableDate",
                "date": "2025-09-13"
            }
        }
    },
    "optionType": "Call"
}


## Creating an Equity Option Definition Instrument

In [3]:
definition_instrument = OptionDefinitionInstrument(
    definition=definition
)

print(js.dumps(definition_instrument.as_dict(), indent=4))


{
    "definition": {
        "underlying": {
            "code": "ASML.AS",
            "underlyingType": "Equity"
        },
        "exercise": {
            "strike": 150,
            "exerciseStyle": "European",
            "schedule": {
                "endDate": {
                    "dateType": "AdjustableDate",
                    "date": "2025-09-13"
                }
            }
        },
        "optionType": "Call"
    }
}


## Creating an Equity Option Pricing Parameters

In [4]:
pricing_parameters=OptionPricingParameters(
    pricing_model="BlackScholes",
    underlying_price_side="Mid",
    option_price_side="Mid",
    valuation_date=date(2024, 12, 31)
)

print(js.dumps(pricing_parameters.as_dict(), indent=4))


{
    "pricingModel": "BlackScholes",
    "underlyingPriceSide": "Mid",
    "optionPriceSide": "Mid",
    "valuationDate": "2024-12-31"
}


## Valuing an Equity Option

In [5]:
response = value(
    definitions=[definition_instrument],
    pricing_preferences=pricing_parameters
)

print(js.dumps(response.as_dict(), indent=4))


{
    "definitions": [
        {
            "definition": {
                "underlying": {
                    "underlyingType": "Equity",
                    "code": "ASML.AS"
                },
                "exercise": {
                    "strike": 150.0,
                    "schedule": {
                        "endDate": {
                            "dateType": "AdjustableDate",
                            "date": "2025-09-13"
                        }
                    },
                    "exerciseStyle": "European"
                },
                "optionType": "Call"
            }
        }
    ],
    "pricingPreferences": {
        "pricingModel": "BlackScholes",
        "underlyingPriceSide": "Mid",
        "optionPriceSide": "Mid",
        "valuationDate": "2024-12-31"
    },
    "analytics": [
        {
            "description": {
                "instrumentCode": "",
                "underlyingRic": "ASML.AS",
                "instrumentDescription": "Cash_E

## Displaying results

In [6]:
valuation = response.analytics[0].valuation

def flatten(data):
    rows = []

    for key, section in data.items():
        row = {"metric": key}

        # Start with raw values
        value = section.get("value")
        unit = section.get("unit")
        percent = section.get("percent")

        # Handle dealCurrency override
        dc = section.get("dealCurrency")
        if dc is not None:
            value = getattr(dc, "value", value)
            unit = getattr(dc, "currency", unit)

        # Round and assign
        if value is not None:
            row["value"] = round(value, 3)
        if unit is not None:
            row["unit"] = unit
        if percent is not None:
            row["percent"] = round(percent, 3)

        rows.append(row)

    return pd.DataFrame(rows).set_index("metric")


flatten(valuation)

,value,unit,percent
metric,,,
volatility,55.809,Percentage,NaN
marketValue,526.805,EUR,NaN
totalMarketValue,526.805,EUR,NaN
intrinsicValue,528.350,EUR,NaN
timeValue,-1.545,EUR,NaN
premiumOverCash,-1.545,EUR,-0.228
moneyness,452.233,Percentage,NaN
annualizedYield,-0.003,Absolute,NaN
